In [1]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os,sys
from keras.models import Model
import csv

# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 128
width = 16
channels = 1
classes = 2
n_inputs = height * width*channels

for i in range(classes) :
    path = "C:/Users/Yi/Desktop/channel-cla/Train/{0}/".format(i)
    print(path)
    Class=os.listdir(path)
    # for a in Class:
    try:
        # image=cv2.imread(path+a)
        file_name = os.path.join(path, 'pdata{0}_128x16x1000_2dB.csv'.format(i+1))
        with open(file_name) as f:
            reader = csv.reader(f)
            csv_file = list(reader)
        csv_file_mat = np.array(csv_file)
        csv_shape = np.shape(csv_file_mat)
        csv_file_mat = np.reshape(csv_file_mat, (csv_shape[0], csv_shape[1], channels))
        num_obj = int(csv_shape[0] / 128)
        for idx in range(num_obj):
            obj = csv_file_mat[int(128 * idx) : int(128 * (idx + 1))]
            # print(type(obj))
            # print(np.shape(obj))
            # exit()
            # image_from_array = Image.fromarray(image, 'RGB')
            # size_image = image_from_array.resize((height, width))
            data.append(obj)
            labels.append(i)
        # print(labels)
    except AttributeError:
        print(" ")
            
Cells=np.array(data)
labels=np.array(labels)

#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
#X_train = X_train.astype('float32')/255 
#X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]

#Randomize the order of the input images
s=np.arange(X_train.shape[0])
np.random.seed(2)
np.random.shuffle(s)
X_train=X_train[s]
y_train=y_train[s]
# print(Cells)
# print(labels)
# print(data)

#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
y_train = to_categorical(y_train, classes)
y_val = to_categorical(y_val, classes)



Using TensorFlow backend.


C:/Users/Yi/Desktop/channel-cla/Train/0/
C:/Users/Yi/Desktop/channel-cla/Train/1/


In [4]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation

from keras.utils.generic_utils import get_custom_objects
def piecewise5(X):
                   return bk.switch(X < -0.6, (0.01 * X ),
                                   bk.switch(X < -0.2, (0.2 * X ),
                                            bk.switch(X < 0.2, (1 * X ),
                                                     bk.switch(X < 0.6, (1.5 * X ),
                                                              bk.switch(X < 5, (3 * X ), (3 * X )))))) 
    
get_custom_objects().update({'piecewise5': Activation(piecewise5)})




input_shape=X_train.shape[1:]

def custom_network(input_shape):

#   model = Sequential()
#   model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
#   model.add(Conv2D(64, (3, 3), activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))
#   model.add(Flatten())
#   model.add(Dense(128, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(num_classes, activation='softmax'))
  
  
    input_img = Input(shape = (128, 16, 1))
    
    conv_1 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(input_img)
    block1_output = GlobalAveragePooling2D()(conv_1)
    max_pool_1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same')(conv_1)
    dropout_1 = Dropout(0.25)(max_pool_1)
    
    
    
    conv_2 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_1)
    block2_output = GlobalAveragePooling2D()(conv_2)
    max_pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_2)
    dropout_2 = Dropout(0.01)(max_pool_2)
    
    
    conv_3 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_2)
    block3_output = GlobalAveragePooling2D()(conv_3)
    
    

   
    
    output = keras.layers.concatenate([block1_output, block2_output, block3_output], axis = 1)
#     output = Flatten()(output)
    output = Dense(64,activation='piecewise5')(output)
    out    = Dense(2, activation='softmax')(output)
    
   

    
    
    
    
    model = Model(inputs = input_img, outputs = out)
    print(model.summary())
    return model
#     input_img = Input(shape = (30, 30, 3))
#     tower_1 = Conv2D(16, (1,1), padding='same', activation='elu')(input_img)
#     tower_x = Conv2D(32, (3,3), padding='same', activation='elu')(tower_1)
#     tower_y = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_x)
#     tower_y = Dropout(0.1)(tower_y)
#     tower_z = Conv2D(32, (1,1), padding='same', activation='elu')(tower_y)
#     tower_a = Conv2D(32, (3,3), padding='same', activation='elu')(tower_z)
#     tower_a = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_a)
#     tower_a = Dropout(0.1)(tower_a)
    
  
#     tower_2 = MaxPooling2D(pool_size=(4, 4), padding='same')(tower_x)
#     tower_2 = Dropout(0.1)(tower_2)
    
#     tower_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_z)
#     tower_3 = Dropout(0.1)(tower_3)
    
    

    
    
    
    
    
    
#     output = keras.layers.concatenate([tower_a, tower_2, tower_3], axis = 1)
#     output = Flatten()(output)

#     out    = Dense(43, activation='softmax')(output)
    
   

    
    
    
    
#     model = Model(inputs = input_img, outputs = out)
#     print(model.summary())
#     return model





sgd=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
model = custom_network(input_shape)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# )
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))

# score = model.evaluate(x_test, y_test, verbose=0)
# score2=model.evaluate(x_train,y_train, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# print('Training loss:', score2[0])
# print('Training accuracy:', score2[1])# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""



Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 16, 1)   0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 128, 16, 64)  640         input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 64, 8, 64)    0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 64, 8, 64)    0           max_pooling2d_5[0][0]            
____________________________________________________________________________________________

'\nSpyder Editor\n\nThis is a temporary script file.\n'

In [ ]:
print(get_flops(model))

In [ ]:
import matplotlib.pyplot as plt



# Plot training & validation accuracy values
plt.plot(hist1.history['accuracy'])
plt.plot(hist1.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.93, 1.0)
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
# plt.plot(hist4.history['loss'])
# plt.plot(hist4.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [ ]:
#max average pool with bias



# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation
from keras import initializers

import tensorflow as tf



def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=bk.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops


from keras.utils.generic_utils import get_custom_objects
def piecewise5(X):
                   return bk.switch(X < -0.6, (0.01 * X ),
                                   bk.switch(X < -0.2, (0.2 * X ),
                                            bk.switch(X < 0.2, (1 * X ),
                                                     bk.switch(X < 0.6, (1.5 * X ),
                                                              bk.switch(X < 5, (3 * X ), (3 * X )))))) 
    
get_custom_objects().update({'piecewise5': Activation(piecewise5)})




input_shape=X_train.shape[1:]

def custom_network(input_shape):

#   model = Sequential()
#   model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
#   model.add(Conv2D(64, (3, 3), activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))
#   model.add(Flatten())
#   model.add(Dense(128, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(num_classes, activation='softmax'))
  
  
#     input_img = Input(shape = (30, 30, 3))
    
#     conv_1 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(input_img)
#     block1_output = GlobalAveragePooling2D()(conv_1)
#     max_pool_1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same')(conv_1)
#     dropout_1 = Dropout(0.25)(max_pool_1)
    
    
    
#     conv_2 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_1)
#     block2_output = GlobalAveragePooling2D()(conv_2)
#     max_pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_2)
#     dropout_2 = Dropout(0.01)(max_pool_2)
    
    
#     conv_3 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_2)
#     block3_output = GlobalAveragePooling2D()(conv_3)
    
    

   
    
#     output = keras.layers.concatenate([block1_output, block2_output, block3_output], axis = 1)
# #     output = Flatten()(output)
#     output = Dense(64,activation='piecewise5')(output)
#     out    = Dense(43, activation='softmax')(output)
    
   

    
    
    
    
#     model = Model(inputs = input_img, outputs = out)
#     print(model.summary())
#     return model



    input_img = Input(shape = (30, 30, 3))
    tower_1 = Conv2D(16, (1,1), padding='same', activation='elu')(input_img)
    tower_x = Conv2D(32, (3,3), padding='same', activation='elu')(tower_1)
    block1_output = GlobalAveragePooling2D()(tower_1)
    tower_y = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_x)
    tower_y = Dropout(0.1)(tower_y)
    tower_z = Conv2D(32, (1,1), padding='same', activation='elu')(tower_y)
    tower_a = Conv2D(32, (3,3), padding='same', activation='elu')(tower_z)
    tower_a = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_a)
    tower_a = Dropout(0.1)(tower_a)
    
  
    tower_2 = AveragePooling2D(pool_size=(4, 4), padding='same')(tower_x)
    tower_2 = Dropout(0.1)(tower_2)
    
    tower_3 = AveragePooling2D(pool_size=(2, 2), padding='same')(tower_z)
    tower_3 = Dropout(0.1)(tower_3)
    
    

    
    
    
    
    
    
    output = keras.layers.concatenate([tower_a, tower_2, tower_3], axis = 1)
    output = Flatten()(output)
    out1 = keras.layers.concatenate([output,block1_output],axis=1)

    out    = Dense(43, activation='softmax')(out1)
    
   

    
    
    
    
    model = Model(inputs = input_img, outputs = out)
    print(model.summary())
    return model





sgd=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
model = custom_network(input_shape)




model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# 
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
hist2 = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))

score3 = model.evaluate(X_val, y_val, verbose=0)
score4=model.evaluate(X_train,y_train, verbose=0)
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])
print('Training loss:', score4[0])
print('Training accuracy:', score[1])


In [ ]:
# Plot training & validation accuracy values
plt.subplot(211)
plt.plot(hist1.history['accuracy'])
plt.plot(hist2.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.98, 1.0)
plt.xlabel('Epoch')
plt.legend(['Triple Pool with Bias', 'Triple Pool without Bias'], loc='upper left')
plt.show()





plt.subplot(212)
plt.plot(hist1.history['val_accuracy'])
plt.plot(hist2.history['val_accuracy'])
plt.title('Model Validation')
plt.ylabel('Validation')
plt.ylim(0.93, 1.0)
plt.xlabel('Epoch')
plt.legend(['Triple Pool with Bias', 'Triple Pool without Bias'], loc='upper left')
plt.show()

In [ ]:
#Definition of the DNN model



# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation

from keras.utils.generic_utils import get_custom_objects
def piecewise5(X):
                   return bk.switch(X < -0.6, (0.01 * X ),
                                   bk.switch(X < -0.2, (0.2 * X ),
                                            bk.switch(X < 0.2, (1 * X ),
                                                     bk.switch(X < 0.6, (1.5 * X ),
                                                              bk.switch(X < 5, (3 * X ), (3 * X )))))) 
    
get_custom_objects().update({'piecewise5': Activation(piecewise5)})




input_shape=X_train.shape[1:]

def custom_network(input_shape):

#   model = Sequential()
#   model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
#   model.add(Conv2D(64, (3, 3), activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))
#   model.add(Flatten())
#   model.add(Dense(128, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(num_classes, activation='softmax'))
  
  
    input_img = Input(shape = (30, 30, 3))
    
    conv_1 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(input_img)
    block1_output = GlobalAveragePooling2D()(conv_1)
    max_pool_1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same')(conv_1)
    dropout_1 = Dropout(0.25)(max_pool_1)
    
    
    
    conv_2 = Conv2D(128, (3,3), padding='same', activation='piecewise5')(dropout_1)
    block2_output = GlobalAveragePooling2D()(conv_2)
    max_pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_2)
    dropout_2 = Dropout(0.01)(max_pool_2)
    
    
    conv_3 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_2)
    block3_output = GlobalAveragePooling2D()(conv_3)
    max_pool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_3)
    dropout_3 = Dropout(0.01)(max_pool_3)
    
    
    conv_4 = Conv2D(128, (3,3), padding='same', activation='piecewise5')(dropout_3)
    block4_output = GlobalAveragePooling2D()(conv_4)
    max_pool_4 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_4)
    dropout_4 = Dropout(0.01)(max_pool_4)
    
    
    conv_5 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_4)
    block5_output = GlobalAveragePooling2D()(conv_5)
    max_pool_5 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_5)
    dropout_5 = Dropout(0.01)(max_pool_5)
    
    
    
    conv_6 = Conv2D(128, (3,3), padding='same', activation='piecewise5')(dropout_5)
    block6_output = GlobalAveragePooling2D()(conv_6)
    
    

   
    
    output = keras.layers.concatenate([block1_output, block2_output, block3_output,block4_output,block5_output, block6_output ], axis = 1)
#     output = Flatten()(output)
    output = Dense(64,activation='piecewise5')(output)
    out    = Dense(43, activation='softmax')(output)
    
   

    
    
    
    
    model = Model(inputs = input_img, outputs = out)
    print(model.summary())
    return model
#     input_img = Input(shape = (30, 30, 3))
#     tower_1 = Conv2D(16, (1,1), padding='same', activation='elu')(input_img)
#     tower_x = Conv2D(32, (3,3), padding='same', activation='elu')(tower_1)
#     tower_y = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_x)
#     tower_y = Dropout(0.1)(tower_y)
#     tower_z = Conv2D(32, (1,1), padding='same', activation='elu')(tower_y)
#     tower_a = Conv2D(32, (3,3), padding='same', activation='elu')(tower_z)
#     tower_a = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_a)
#     tower_a = Dropout(0.1)(tower_a)
    
  
#     tower_2 = MaxPooling2D(pool_size=(4, 4), padding='same')(tower_x)
#     tower_2 = Dropout(0.1)(tower_2)
    
#     tower_3 = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_z)
#     tower_3 = Dropout(0.1)(tower_3)
    
    

    
    
    
    
    
    
#     output = keras.layers.concatenate([tower_a, tower_2, tower_3], axis = 1)
#     output = Flatten()(output)

#     out    = Dense(43, activation='softmax')(output)
    
   

    
    
    
    
#     model = Model(inputs = input_img, outputs = out)
#     print(model.summary())
#     return model





sgd=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
model = custom_network(input_shape)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# )
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
hist3 = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))

# score = model.evaluate(x_test, y_test, verbose=0)
# score2=model.evaluate(x_train,y_train, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# print('Training loss:', score2[0])
# print('Training accuracy:', score2[1])


score5 = model.evaluate(X_val, y_val, verbose=0)
score6=model.evaluate(X_train,y_train, verbose=0)
print('Test loss:', score6[0])
print('Test accuracy:', score6[1])
print('Training loss:', score6[0])
print('Training accuracy:', score6[1])


In [ ]:
# Plot training & validation accuracy values
import matplotlib.pyplot as plt
plt.subplot(211)
plt.plot(hist1.history['accuracy'])
plt.plot(hist3.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.98, 1.0)
plt.xlabel('Epoch')
plt.legend(['Triple Pool with Bias', 'Triple Pool without Bias'], loc='upper left')
plt.show()

In [ ]:
#max average pool with bias and adam optimizer



# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation
from keras import initializers

import tensorflow as tf



def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=bk.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops


from keras.utils.generic_utils import get_custom_objects
def piecewise5(X):
                   return bk.switch(X < -0.6, (0.01 * X ),
                                   bk.switch(X < -0.2, (0.2 * X ),
                                            bk.switch(X < 0.2, (1 * X ),
                                                     bk.switch(X < 0.6, (1.5 * X ),
                                                              bk.switch(X < 5, (3 * X ), (3 * X )))))) 
    
get_custom_objects().update({'piecewise5': Activation(piecewise5)})




input_shape=X_train.shape[1:]

def custom_network(input_shape):

#   model = Sequential()
#   model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
#   model.add(Conv2D(64, (3, 3), activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2)))
#   model.add(Dropout(0.25))
#   model.add(Flatten())
#   model.add(Dense(128, activation='relu'))
#   model.add(Dropout(0.5))
#   model.add(Dense(num_classes, activation='softmax'))
  
  
#     input_img = Input(shape = (30, 30, 3))
    
#     conv_1 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(input_img)
#     block1_output = GlobalAveragePooling2D()(conv_1)
#     max_pool_1 = MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding='same')(conv_1)
#     dropout_1 = Dropout(0.25)(max_pool_1)
    
    
    
#     conv_2 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_1)
#     block2_output = GlobalAveragePooling2D()(conv_2)
#     max_pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same')(conv_2)
#     dropout_2 = Dropout(0.01)(max_pool_2)
    
    
#     conv_3 = Conv2D(64, (3,3), padding='same', activation='piecewise5')(dropout_2)
#     block3_output = GlobalAveragePooling2D()(conv_3)
    
    

   
    
#     output = keras.layers.concatenate([block1_output, block2_output, block3_output], axis = 1)
# #     output = Flatten()(output)
#     output = Dense(64,activation='piecewise5')(output)
#     out    = Dense(43, activation='softmax')(output)
    
   

    
    
    
    
#     model = Model(inputs = input_img, outputs = out)
#     print(model.summary())
#     return model



    input_img = Input(shape = (30, 30, 3))
    tower_1 = Conv2D(16, (1,1), padding='same', activation='elu', bias_initializer=initializers.Constant(.1))(input_img)
    tower_x = Conv2D(32, (3,3), padding='same', activation='elu', bias_initializer=initializers.Constant(.1))(tower_1)
    block1_output = GlobalAveragePooling2D()(tower_1)
    tower_y = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_x)
    tower_y = Dropout(0.1)(tower_y)
    tower_z = Conv2D(32, (1,1), padding='same', activation='elu', bias_initializer=initializers.Constant(.1))(tower_y)
    tower_a = Conv2D(32, (3,3), padding='same', activation='elu', bias_initializer=initializers.Constant(.1))(tower_z)
    tower_a = MaxPooling2D(pool_size=(2, 2), padding='same')(tower_a)
    tower_a = Dropout(0.1)(tower_a)
    
  
    tower_2 = AveragePooling2D(pool_size=(4, 4), padding='same')(tower_x)
    tower_2 = Dropout(0.1)(tower_2)
    
    tower_3 = AveragePooling2D(pool_size=(2, 2), padding='same')(tower_z)
    tower_3 = Dropout(0.1)(tower_3)
    
    

    
    
    
    
    
    
    output = keras.layers.concatenate([tower_a, tower_2, tower_3], axis = 1)
    output = Flatten()(output)
    out1 = keras.layers.concatenate([output,block1_output],axis=1)

    out    = Dense(43, activation='softmax')(out1)
    
   

    
    
    
    
    model = Model(inputs = input_img, outputs = out)
    print(model.summary())
    return model





sgd=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
model = custom_network(input_shape)



model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# 
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
hist4 = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))

score7 = model.evaluate(X_val, y_val, verbose=0)
score8=model.evaluate(X_train,y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Training loss:', score2[0])
print('Training accuracy:', score2[1])

In [ ]:
# Plot training & validation accuracy values
import matplotlib.pyplot as plt
plt.subplot(211)
# plt.plot(hist1.history['accuracy'])
plt.plot(hist3.history['accuracy'])
plt.plot(hist4.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
# plt.ylim(0.98, 1.0)
plt.xlabel('Epoch')
plt.legend(['Triple Pool with Bias SGD', 'GC Net without bias','Triple Pool with Bias Adam'], loc='lower right')
plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='accurate.png',show_shapes=True, show_layer_names=True,expand_nested=True)

In [ ]:
#Max pool with Bias



# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Dropout(rate=0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(rate=0.5))
# model.add(Dense(43, activation='softmax'))
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation
from keras import initializers



from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


import tensorflow as tf



def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=bk.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops




from keras.utils.generic_utils import get_custom_objects
def piecewise5(X):
                   return bk.switch(X < -0.6, (0.01 * X ),
                                   bk.switch(X < -0.2, (0.2 * X ),
                                            bk.switch(X < 0.2, (1 * X ),
                                                     bk.switch(X < 0.6, (1.5 * X ),
                                                              bk.switch(X < 5, (3 * X ), (3 * X )))))) 
    
get_custom_objects().update({'piecewise5': Activation(piecewise5)})




input_shape=X_train.shape[1:]

def custom_network(input_shape):

    input_img = Input(shape = (30, 30, 3))
    x=Conv2D(64,(3,3), padding='same', activation='relu')(input_img)
    x=Conv2D(64,(3,3), padding='same', activation='relu')(x)
    x=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    
    
    x=Conv2D(128,(3,3),padding='same',activation='relu')(x)
    x=Conv2D(128,(3,3),padding='same',activation='relu')(x)
    x=MaxPooling2D(pool_size=(2,2), strides=(2,2),padding='same')(x)
    
    
    x=Conv2D(256,(3,3),padding='same',activation='relu')(x)
    x=Conv2D(256,(3,3),padding='same',activation='relu')(x)
    x=Conv2D(256,(3,3),padding='same',activation='relu')(x)
    x=MaxPooling2D(pool_size=(2,2), strides=(2,2),padding='same')(x)
             
    
    x = Flatten()(x)
    x = Dense(512,activation='piecewise5')(x)
    x = Dense(43, activation='softmax')(x)
    
    model = Model(inputs = input_img, outputs = x)
    print(model.summary())
    return model






# sgd=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
# model = custom_network(input_shape)






# base_model = InceptionV3(weights='imagenet', include_top=False)

# # add a global spatial average pooling layer
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# # let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(200, activation='softmax')(x)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)







from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras.applications.vgg16 import VGG16

input_img = Input(shape=(30, 30, 3))
# Generate a model with all layers (with top)
resnet = ResNet50(input_tensor=input_img, include_top=True)

#Add a layer where input is the output of the  second last layer 
x = Dense(43, activation='softmax', name='predictions')(resnet.layers[-2].output)

#Then create the corresponding model 
model = Model(input=resnet.input, output=x)
print(model.summary())



# this could also be the output a different Keras model or layer
# input_img = Input(shape=(30, 30, 3))  # this assumes K.image_data_format() == 'channels_last'

# base_model = ResNet50(input_tensor=input_img)
# x = base_model.output
# predictions = Dense(43, activation='softmax')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# # model = ResNet50(input_shape)
# # base_model = ResNet50(input_shape)
# # model = Model(inputs=base_model.input)



# print(model.summary())



#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training












model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

# #Compilation of the model
# model.compile(
#     loss='categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy']
# )
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 20
hist5 = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))



score7 = model.evaluate(X_val, y_val, verbose=0)
score8=model.evaluate(X_train,y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Training loss:', score2[0])
print('Training accuracy:', score2[1])